In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

In [3]:
def reduce_mem(df: pd.DataFrame):
    "This method reduces memory for numeric columns in the dataframe";

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', "uint16", "uint32", "uint64"];
    start_mem = df.memory_usage().sum() / 1024**2;

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min();
            c_max = df[col].max();

            if "int" in str(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                if c_min >= np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"Start - end memory:- {start_mem:5.2f} - {end_mem:5.2f} Mb");
    return df;

In [4]:
train = pd.read_csv("./playground-series-s4e7/train.csv")
target = pd.read_csv("./playground-series-s4e7/test.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [6]:
train = train.drop(columns=["id"], axis=1)

In [7]:
target= target.drop(columns=["id"], axis=1)

In [8]:
category_columns = train.select_dtypes(include="object").columns
category_columns = category_columns.tolist()

def label_encoding(df):
    # Instance of LabelEncoder
    label_encoders = {col: LabelEncoder() for col in category_columns}

    # Label Encoding
    for col in category_columns:
    
        df[col] = label_encoders[col].fit_transform(df[col])

label_encoding(train)
label_encoding(target)

In [9]:
train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,21,1,35.0,0,0,1,65101.0,124.0,187,0
1,1,43,1,28.0,0,2,1,58911.0,26.0,288,1
2,0,25,1,14.0,1,1,0,38043.0,152.0,254,0
3,0,35,1,1.0,0,0,1,2630.0,156.0,76,0
4,0,36,1,15.0,1,0,0,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...
11504793,1,48,1,6.0,0,0,1,27412.0,26.0,218,0
11504794,0,26,1,36.0,0,1,1,29509.0,152.0,115,1
11504795,0,29,1,32.0,1,1,0,2630.0,152.0,189,0
11504796,0,51,1,28.0,0,0,1,48443.0,26.0,274,1


In [10]:
target

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0,20,1,47.0,0,1,0,2630.0,160.0,228
1,1,47,1,28.0,0,0,1,37483.0,124.0,123
2,1,47,1,43.0,0,0,1,2630.0,26.0,271
3,0,22,1,47.0,1,1,0,24502.0,152.0,115
4,1,51,1,19.0,0,0,0,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28.0,0,0,1,51661.0,124.0,109
7669862,1,28,1,50.0,1,1,0,25651.0,152.0,184
7669863,1,47,1,33.0,1,0,0,2630.0,138.0,63
7669864,1,30,1,28.0,0,1,1,38866.0,124.0,119


In [11]:
train = reduce_mem(train)

Start - end memory:- 833.86 - 230.41 Mb


In [12]:
target = reduce_mem(target)

Start - end memory:- 497.39 - 146.29 Mb


In [13]:
b=train['Response'][train['Response']==1].count()

In [14]:
a=train['Response'][train['Response']==0].count()

In [15]:
b/len(train)

0.12299729208631043

In [16]:
a/len(train)

0.8770027079136896

In [17]:
train.isna().sum()

Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [18]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [19]:
train.describe()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000,11504798.000
mean,0.541,38.384,0.998,26.419,0.463,0.522,0.503,30461.359,112.425,163.898,0.123
std,0.498,14.993,0.044,12.992,0.499,0.577,0.500,16454.744,54.036,79.980,0.328
min,0.000,20.000,0.000,0.000,0.000,0.000,0.000,2630.000,1.000,10.000,0.000
25%,0.000,24.000,1.000,15.000,0.000,0.000,0.000,25277.000,29.000,99.000,0.000
50%,1.000,36.000,1.000,28.000,0.000,0.000,1.000,31824.000,151.000,166.000,0.000
75%,1.000,49.000,1.000,35.000,1.000,1.000,1.000,39451.000,152.000,232.000,0.000
max,1.000,85.000,1.000,52.000,1.000,2.000,1.000,540165.000,163.000,299.000,1.000


In [20]:
x = train.drop(columns=['Response'],axis=1)
y = train["Response"]

In [22]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [23]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, max_depth=3,verbose=1)

In [25]:
gbc.fit(X_train,y_train)

      Iter       Train Loss   Remaining Time 
         1           0.7424           29.07m
         2           0.7392           28.27m
         3           0.7361           27.88m
         4           0.7330           27.68m
         5           0.7300           27.40m
         6           0.7271           27.24m
         7           0.7242           27.15m
         8           0.7214           27.16m
         9           0.7187           27.21m
        10           0.7160           27.06m
        20           0.6918           24.40m
        30           0.6718           22.47m
        40           0.6550           19.81m
        50           0.6408           16.80m
        60           0.6286           13.63m
        70           0.6180           10.20m
        80           0.6089            6.80m
        90           0.6010            3.40m
       100           0.5938            0.00s


GradientBoostingClassifier(learning_rate=0.01, verbose=1)

In [29]:
val_result = gbc.predict(X_val)

In [30]:
val_result

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [32]:
roc_auc_score(y_val,val_result)

0.5

In [33]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(y_val, val_result))

C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.88      1.00      0.93   2017948
           1       0.00      0.00      0.00    283012

    accuracy                           0.88   2300960
   macro avg       0.44      0.50      0.47   2300960
weighted avg       0.77      0.88      0.82   2300960



C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
a=gbc.predict(target)

In [ ]:
submit = pd.read_csv("./playground-series-s4e7/sample_submission.csv")
pred = a 
submit["Response"] = pred
submit.to_csv("submission_GBC.csv", index=False)
submit.head()